In [1]:
import numpy as np
import pandas as pd 

dataset = pd.read_csv('../dataset/dados_pregao_lwsa3.csv')
colunas = ['preco_ultimo_negocio']
data = dataset[colunas]
train_size = int(len(data) * 0.8)
train_data, test_data = data[:train_size], data[train_size:]

train_data = train_data.values
test_data = test_data.values

data

,preco_ultimo_negocio
0,65.34
1,63.85
2,63.84
3,66.56
4,67.84
...,...
1134,69.00
1135,67.65
1136,67.27
1137,68.06


In [2]:
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length])
    return np.array(X), np.array(y)

sequence_length = 10
X_train, y_train = create_sequences(train_data, sequence_length)
X_test, y_test = create_sequences(test_data, sequence_length)

In [3]:
from keras.models import Sequential, save_model, load_model
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(sequence_length, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=100, batch_size=32)

predictions = model.predict(X_test)

mse = np.mean((predictions - y_test) ** 2)
print("Mean Squared Error:", mse)

Epoch 1/100


e:\Workspace\pregao_forecasting\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 337.3427
Epoch 2/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 66.8604
Epoch 3/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 53.3304
Epoch 4/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 71.7049
Epoch 5/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 49.9474
Epoch 6/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 42.0857
Epoch 7/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 42.6152
Epoch 8/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 51.8348
Epoch 9/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 47.6563
Epoch 10/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 46.9061
Epoch 11/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 39.2412
Epoch 12/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 35.7172
Epoch 13/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 48.5552
Epoch 14/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 35.6863
Epoch 15/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 30.2172


In [4]:
test_predictions1 = model.predict(X_test).flatten()

X_test_list = []
for i in range(len(X_test)):
    X_test_list.append(X_test[i][0])
    

test_predictions_df1 = pd.DataFrame({'X_test':list(X_test_list), 
                                    'LSTM Prediction':list(test_predictions1)})

test_predictions_df1

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


,X_test,LSTM Prediction
0,[4.52],4.322242
1,[4.38],4.323106
2,[4.32],4.326880
3,[4.45],4.377225
4,[4.45],4.440058
...,...,...
213,[70.6],67.708893
214,[69.15],72.252808
215,[74.11],76.398300
216,[73.97],78.807014
